# Main Historical Extraction Engine
Run as many times as needed to get full list of players. 

Warning: Full extract takes >3 hours

In [75]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [76]:
# Get all files in working data directory
path = os.getcwd()
path = path.replace("notebooks\\basketballProject\\extraction","data\\basketballProject")+'\\'+'historical-extraction-files\\'
csv_files = glob.glob(os.path.join(path,"*.csv"))
write_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/historical-extraction-files/nba_data_"
# write_path = "../../data/basketballProject/historical-extraction-files/nba_data_"

In [77]:
# Check what players you have already analyzed to avoid duplication
check_df = pd.DataFrame()
for file in csv_files:

    df = pd.read_csv(file)
    frames = [check_df, df]
    check_df = pd.concat(frames)

In [78]:
try:
    unique_ids = check_df.PLAYER_ID.unique()
except:
    pass

In [79]:
# import teams, players, and sort them into active or all players
from nba_api.stats.static import teams, players
nba_players = players.get_players()
# active_players = players.get_active_players()
teams = teams.get_teams()

In [80]:
# import use-case specific functions
from nba_api.stats.endpoints import playercareerstats

In [81]:
# Add entry for each player on whether they have been analyzed or not. 
# Default is not analyzed
for player in nba_players:
    player['Parsed ?'] = False
    try:
        if player["id"] in unique_ids:
            player["Parsed ?"] = True
    except:
        pass

In [82]:
# Initialize collection dataframe
master_list = pd.DataFrame()

# Create temp dataframe and initialize count to serve as a staging 
# Dataframe and break up the merging into small batches
temp_df = pd.DataFrame()
count = 0

In [83]:
# Parse thru each active player
first_id = 0
last_id = 0
try:
    for player in nba_players:

        # Check if they have been already analyzed
        if player['Parsed ?'] == True:
            pass
        
        # If they haven't, proceed
        else:
            passedId = player['id']
            player_name = player["full_name"]
            # Get player stats based on their ID
            career_stats = playercareerstats.PlayerCareerStats(player_id = passedId).get_data_frames()[0]
            
            # Get ID of first and last players analyzed for file naming purposes
            if first_id == 0:
                first_id = passedId
            last_id = passedId

            # Add player names to resulting dataframe to keep track of players
            career_stats.insert(1,"full_name",player_name)

            # Append resulting dataframe to temp dataframe
            frames = [temp_df,career_stats]
            temp_df = pd.concat(frames)

            # Set analyzed flag to true
            player['Parsed ?'] = True
            count = count + 1

            # Once enough volume is reached in the temp dataframe, append it to the main and re-initialize the dataframe
            if (count%20) == 0:
                frames = [master_list, temp_df]
                master_list = pd.concat(frames)
                temp_df = pd.DataFrame()
                print("Temp Full - Resetting")
except:
    frames = [master_list, temp_df]
    master_list = pd.concat(frames)
    temp_df = pd.DataFrame()
    print("Temp Full - Resetting")
    print("TimeoutError")

Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full - Resetting
Temp Full 

In [84]:
if first_id == 0:
    print("empty")
    pass
else:
    master_list.to_csv(write_path+str(first_id)+"to"+str(last_id)+".csv")